# Affine Transformation

Affine Transformationen sind Transformationen des Raums, bei denen bestimmte geometrische Eigenschaften erhalten bleiben.
Lose formuliert:

- Punkte einer Geraden werden wieder auf eine Gerade abgebildet
- Parallele Geraden bleiben parallel
- Das Verhältnis dreier Punkte einer Geraden bleibt erhalten

Affine Transformationen kann man mittels Matrixmultiplikation darstellen.

In [ ]:
%matplotlib inline
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import numpy as np

def plot(X, fig, ax):
    ax.scatter(X[:,0], X[:,1], alpha=0.5)
    ax.set_xlabel(r'$x_1$')
    ax.set_ylabel(r'$x_2$')
    ax.axis('equal');


Erzeugen wir uns zunächst mal eine Punktwolke:

In [ ]:
m = 200

rng = np.random.default_rng(50)
X_id = rng.standard_normal(size=(m,2))

fig, ax = plt.subplots(1, figsize=(10,10))
plot(X_id, fig, ax);


Die Punktwolke können wir durch Multiplikation mit einer 2x2 Matrix verzerren:

In [ ]:

theta = np.pi/2
rot = [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
shear = [[1, 2], [0, 1]]
stretchX = [[2, 0],[0, 1]]
stretchY = [[1, 0],[0, 1.5]]

fig, axs = plt.subplots(2,3, figsize=(15,10))

def plotTransformed(label, input, transformation, ax):
    box = np.asarray([[x,y] for x in np.linspace(-4,4, 20) for y in np.linspace(-4,4, 20)]).T
    box_transformed = np.dot(box.T, transformation)
    data = np.dot(input, transformation)
    A = data[:, 0]
    B = data[:, 1]
    ax.axis('equal')
    ax.set(xlim=[-10,10], ylim=[-10, 10])
    ax.set_title(label)
    ax.scatter(box_transformed[:,0], box_transformed[:,1], s=1, c='grey', alpha=0.5);
    ax.scatter(A, B, alpha=0.6);

plotTransformed('Identität', X_id, np.identity(2), ax=axs[0, 0])
plotTransformed('Rotation', X_id, rot, ax=axs[0, 1])
plotTransformed('Shear', X_id, shear, ax=axs[0, 2])
plotTransformed('Stretch X', X_id, stretchX, ax=axs[1, 0])
plotTransformed('Stretch Y', X_id, stretchY, ax=axs[1, 1])

combined = np.dot(np.dot(stretchX, stretchY), np.dot(rot, shear))
plotTransformed('Combined', X_id, combined, ax=axs[1, 2])


Nehmen wir doch mal die letzte Punktwolke und ermitteln dafür im folgenden die Kovarianzmatrix und deren Eigenvektoren.

Hier also unsere Punktwolke nochmal größer:

In [ ]:
X = np.dot(X_id, combined)

fig, ax = plt.subplots(1, figsize=(10,10))
plotTransformed('Verzerrte Punktwolke', X_id, combined, ax=ax);
ax.set(xlim=[-12, 12], ylim=[-12, 12]);
X.shape

## Skalierung auf Standard

Wir standardisieren die Features durch Entfernen des Mittelwerts und Skalierung auf Einheitsvarianz.

Der Standardwert einer Stichprobe x wird berechnet als:

$$z=\frac{x-\mu}{\sigma}$$

wobei $\mu$ der Mittelwert der Trainingsstichproben und $\sigma$ die Standardabweichung der Trainingsstichproben ist.

Zentrierung und Skalierung erfolgen unabhängig für jedes Merkmal, indem die relevanten Statistiken über die Stichproben im Trainingsset berechnet werden. Mittelwert und Standardabweichung werden dann gespeichert, um bei späteren Daten mittels Transformation verwendet zu werden.

Die Standardisierung eines Datensatzes ist eine häufige Anforderung für viele Schätzer des maschinellen Lernens: Sie könnten sich schlecht verhalten, wenn die einzelnen Merkmale nicht mehr oder weniger wie normalverteilte Standarddaten aussehen (z.B. Gaußsch mit Mittelwert 0 und Einheitsvarianz).

Beispielsweise gehen viele Elemente, die in der Cost-Function eines Lernalgorithmus verwendet werden (wie die L1- und L2-Regularisierer von linearen Modellen) davon aus, dass alle Merkmale um 0 zentriert sind und Varianzen in gleicher Größenordnung haben. Wenn ein Merkmal eine Varianz hat, die um Größenordnungen größer ist als andere, könnte es die Zielfunktion dominieren und den Schätzer unfähig machen, von anderen Merkmalen zu lernen.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
print(f'Skalierer: {scaler}')

X_std = scaler.transform(X)

fig, ax = plt.subplots(1, figsize=(10,10))
plotTransformed('Skalierte verzerrte Punktwolke', X_std, np.identity(2), ax=ax);
ax.set(xlim=[-4, 4], ylim=[-4, 4]);

## Kovarianzmatrix

In [ ]:
mean_vec = np.mean(X_std, axis=0)
print(f'Means: {mean_vec}')

cov_mat = (X_std - mean_vec).T.dot(X_std - mean_vec) / (X_std.shape[0]-1)

display(Markdown('Kovarianz Matrix: \n\n' + r'$Cov(x_1, x_2)= \begin{bmatrix} \sigma_{1,1}\;\sigma_{1,2} \\ \sigma_{2,1}\;\sigma_{2,2} \end{bmatrix}= \begin{bmatrix}' + '{:.3f}'.format(cov_mat[0,0]) + '\;' + '{:.3f}'.format(cov_mat[0,1]) + r' \\ ' + '{:.3f}'.format(cov_mat[1,0]) + '\,' + '{:.3f}'.format(cov_mat[1,1]) + ' \end{bmatrix}$' % cov_mat))

## Eigenvektoren

Ein **Eigenvektor** einer Matrix ist ein vom Nullvektor verschiedener Vektor, dessen Richtung durch die Abbildung - sprich Multiplikation mit der Matrix - nicht verändert wird. Ein Eigenvektor wird also nur skaliert und man bezeichnet den Skalierungsfaktor als **Eigenwert** der Abbildung.

Mit einer Matrix $A$ und einem Eigenvektor $v\neq0$ gilt dann $$A v = \lambda v$$

Für kleine Matrizen kann man die Eigenvektoren noch exakt ermitteln, ansonsten werden numerische Methoden angewandt. Die Wahl der Methode ist abhängig von der Art der Matrix (insbesondere ob sparse oder dicht besetzt).

> Array Slicing in Python: Array slicing hat folgende Syntax: a[start:stop:step].
>
> Wenn ein Parameter nicht angegeben ist, dann werden folgende Defaults verwendet: 
> - step=1
> - falls step > 0: start=0, stop=length
> - falls step < 0: start=length-1, stop=-1
>
> d.h. der Ausdruck a[::-1] bedeutet: Nimm das gesamte Array, aber durchlaufe es rückwärts.
>
> Hier nutzen wir das, um eine **absteigende Sortierung** zu erhalten.

In [ ]:
eig_vals_, eig_vecs_ = np.linalg.eig(cov_mat)

eig_vals = np.sort(eig_vals_)[::-1]
eig_vecs = eig_vecs_[:, eig_vals_.argsort()[::-1]].T # np.linalg.eig gibt Spalten-Eigenvektoren zurück, daher transponieren

print(f'''
Eigenwerte:
{eig_vals}

Eigenvektoren:
{eig_vecs}
''')

Wir können noch prüfen, ob die Gleichung $Av=\lambda v$ tatsächlich hält:

In [ ]:
print(cov_mat.dot(eig_vecs) - eig_vals*eig_vecs)    

In [ ]:
def draw_vector(v0, v1, ax, c='black'):
    arrowprops=dict(arrowstyle='->',
                    linewidth=2,
                    shrinkA=0, shrinkB=0,
                    color=c
                   )
    ax.annotate('', v1, v0, arrowprops=arrowprops)

for length, vector in zip(eig_vals, eig_vecs): 
    v = vector * 2 * np.sqrt(length)
    draw_vector([0,0], [0.0] + v, ax)
fig

Die Eigenvektoren der Kovarianzmatrix bilden ein gutes Koordinatensystem, welches der Verteilung der Punktwolke folgt.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(21,10))

# zunächst noch mal unsere Punktewolke
plotTransformed('Skalierte verzerrte Punktwolke', X_std, np.identity(2), ax=axs[0]);
axs[0].set(xlim=[-4, 4], ylim=[-4, 4]);

cs = ['green', 'red']
for idx, (length, vector) in enumerate(zip(eig_vals, eig_vecs)):
    v = vector * 2 * np.sqrt(length)
    draw_vector([0,0], [0.0] + v, axs[0], cs[idx])

# und jetzt die Transformation durch PCA
X_pca = X_std.dot(eig_vecs)

plotTransformed('Eigenvektorielle Koordinaten', X_std, eig_vecs, ax=axs[1]);
draw_vector([0,0], [2*np.sqrt(eig_vals[0]), 0], axs[1], cs[0])
draw_vector([0,0], [0, 2*np.sqrt(eig_vals[1])], axs[1], cs[1])

axs[1].set(xlabel='Eigenvektor 1', ylabel='Eigenvektor 2')
axs[1].set(xlim=[-4, 4], ylim=[-4, 4]);


**Und das ist genau der Ansatz der PCA:**

- ermittle ein neues Koordinatensystem, wobei die Koordinaten nach der Größe ihres Beitrags sortiert sind
- wähle dann die ersten k Koordinaten aus, so dass noch hinreichend Information vorhanden ist
- und projiziere dann den Featureraum auf diese k Dimensionen